# 정보처리기사 코드 문제 크롤러 v3.4 (진짜 최종!)

**🎯 문제 범위 정확히 추출!**

## v3.4 수정
- ✅ **문제 범위를 먼저 찾은 후 그 안의 코드만 추출**
- ✅ 다른 문제의 코드가 섞이는 버그 수정
- ✅ 각 문제마다 정확한 코드 매칭

## 작동 원리
1. HTML에서 "12. 다음은" 찾기
2. "13. 다음은"까지의 범위 추출
3. 그 범위 안의 colorscripter-code-table만 파싱
4. 완벽!

## 1. 라이브러리 설치

In [ ]:
!pip install requests beautifulsoup4 lxml

## 2. 라이브러리 임포트

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re
from urllib.parse import urljoin
from google.colab import files

## 3. 크롤러 클래스 정의 (문제 범위 정확히!)

In [ ]:
class TistoryCrawler:
    def __init__(self, category_url):
        self.category_url = category_url
        self.base_url = "https://chobopark.tistory.com"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        self.code_problems = []
        
    def get_post_links(self):
        """카테고리 페이지에서 모든 포스트 링크 가져오기"""
        print("포스트 링크를 수집 중...")
        
        try:
            response = requests.get(self.category_url, headers=self.headers)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')
            
            post_links = []
            
            all_links = soup.find_all('a', href=True)
            for link in all_links:
                title = link.get_text(strip=True)
                if '정보처리기사' in title and '실기' in title and '복원' in title:
                    full_url = urljoin(self.base_url, link['href'])
                    if full_url not in [p['url'] for p in post_links]:
                        year_match = re.search(r'(\d{4})년\s*(\d)회', title)
                        if year_match:
                            post_links.append({
                                'url': full_url, 
                                'title': title,
                                'year': year_match.group(1),
                                'session': year_match.group(2)
                            })
            
            post_links.sort(key=lambda x: (x['year'], x['session']), reverse=True)
            
            print(f"총 {len(post_links)}개의 포스트를 찾았습니다.")
            return post_links
            
        except Exception as e:
            print(f"링크 수집 중 오류: {e}")
            return []
    
    def crawl_post_content(self, post_info):
        """개별 포스트 크롤링"""
        print(f"\n크롤링 중: [{post_info['year']}년 {post_info['session']}회]")
        
        try:
            time.sleep(1)
            response = requests.get(post_info['url'], headers=self.headers)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')
            
            content_area = soup.find('div', class_='tt_article_useless_p_margin') or \
                          soup.find('article') or \
                          soup.find('div', {'class': 'content'})
            
            if not content_area:
                content_area = soup.find('div', id='content') or soup.find('main')
            
            if content_area:
                return {
                    'text': content_area.get_text(separator='\n', strip=True),
                    'html': str(content_area),
                    'year': post_info['year'],
                    'session': post_info['session'],
                    'url': post_info['url']
                }
            else:
                print(f"  ✗ 본문을 찾을 수 없습니다")
                return None
                
        except Exception as e:
            print(f"  ✗ 크롤링 오류: {e}")
            return None
    
    def extract_problem_content(self, text, html, problem_num):
        """특정 문제 번호의 전체 내용 추출"""
        from bs4 import BeautifulSoup
        
        # HTML 파싱
        soup = BeautifulSoup(html, 'html.parser')
        
        # 전체 HTML을 문자열로
        html_str = str(soup)
        
        # 문제 시작 위치 찾기 (HTML에서)
        problem_pattern = rf'{problem_num}\.\s*다음은'
        problem_match = re.search(problem_pattern, html_str)
        
        if not problem_match:
            return None
        
        problem_start_pos = problem_match.start()
        
        # 다음 문제 찾기
        next_num = int(problem_num) + 1
        next_pattern = rf'{next_num}\.\s*다음은'
        next_match = re.search(next_pattern, html_str[problem_start_pos + 10:])
        
        if next_match:
            problem_end_pos = problem_start_pos + 10 + next_match.start()
        else:
            problem_end_pos = len(html_str)
        
        # 문제 범위의 HTML 추출
        problem_html = html_str[problem_start_pos:problem_end_pos]
        
        # 이 범위의 HTML을 다시 파싱
        problem_soup = BeautifulSoup(problem_html, 'html.parser')
        
        # 문제 텍스트 (정리된 버전)
        problem_text = problem_soup.get_text().strip()
        
        # 코드 블록 추출 - Color Scripter 테이블만 찾기
        code_blocks = []
        
        # colorscripter-code-table 찾기 (이제 문제 범위 안에서만!)
        code_tables = problem_soup.find_all('table', class_='colorscripter-code-table')
        
        for table in code_tables:
            # 테이블의 두 번째 td 찾기 (코드가 있는 칸)
            rows = table.find_all('tr')
            for row in rows:
                tds = row.find_all('td')
                if len(tds) >= 2:
                    # 두 번째 td가 코드
                    code_td = tds[1]
                    
                    # 모든 코드 라인 추출
                    code_lines = []
                    for div in code_td.find_all('div', style=lambda x: x and 'padding: 0 6px' in x):
                        line = div.get_text()
                        # HTML 엔티티 변환
                        line = line.replace('&lt;', '<').replace('&gt;', '>')
                        line = line.replace('&amp;', '&').replace('&nbsp;', ' ')
                        line = line.replace('&quot;', '"')
                        code_lines.append(line)
                    
                    if code_lines:
                        code = '\n'.join(code_lines).strip()
                        if len(code) > 20:  # 너무 짧은 건 제외
                            code_blocks.append(code)
        
        return {
            'full_text': problem_text,
            'code_blocks': code_blocks
        }
    
    def clean_html(self, text):
        """HTML 태그 제거"""
        # <br>, <p> 등을 줄바꿈으로
        text = re.sub(r'<br\s*/?>', '\n', text, flags=re.IGNORECASE)
        text = re.sub(r'</p>', '\n', text, flags=re.IGNORECASE)
        
        # 나머지 HTML 태그 제거
        text = re.sub(r'<[^>]+>', '', text)
        
        # HTML 엔티티 변환
        text = text.replace('&lt;', '<').replace('&gt;', '>')
        text = text.replace('&amp;', '&').replace('&nbsp;', ' ')
        text = text.replace('&quot;', '"')
        
        # 연속된 공백/줄바꿈 정리
        text = re.sub(r'\n\s*\n+', '\n\n', text)
        
        return text.strip()
    
    def extract_code_problems(self, content):
        """개별 코드 문제 추출"""
        text = content['text']
        html = content['html']
        year = content['year']
        session = content['session']
        url = content['url']
        
        problems = []
        
        # 언어별 패턴
        language_patterns = {
            'C': [
                r'(\d+)\.\s*다음은\s*C\s*언어.*?문제',
                r'(\d+)\.\s*다음은\s*C언어.*?문제',
            ],
            'Java': [
                r'(\d+)\.\s*다음은\s*Java.*?문제',
                r'(\d+)\.\s*다음은\s*java.*?문제',
                r'(\d+)\.\s*다음은\s*자바.*?문제',
            ],
            'Python': [
                r'(\d+)\.\s*다음은\s*Python.*?문제',
                r'(\d+)\.\s*다음은\s*python.*?문제',
                r'(\d+)\.\s*다음은\s*파이썬.*?문제',
                r'(\d+)\.\s*다음은\s*Pyhon.*?문제',
            ]
        }
        
        found_problems = set()  # 중복 방지
        
        for language, patterns in language_patterns.items():
            for pattern in patterns:
                matches = re.finditer(pattern, text, re.IGNORECASE)
                for match in matches:
                    problem_num = match.group(1)
                    
                    # 중복 체크
                    key = f"{year}-{session}-{problem_num}"
                    if key in found_problems:
                        continue
                    found_problems.add(key)
                    
                    # 문제 전체 내용 추출 (텍스트와 HTML 둘 다 전달)
                    problem_content = self.extract_problem_content(text, html, problem_num)
                    
                    if problem_content:
                        problems.append({
                            'year': year,
                            'session': session,
                            'problem_num': problem_num,
                            'language': language,
                            'title': f"[{year}년 {session}회] {problem_num}번 - {language}",
                            'url': url,
                            'question': problem_content['full_text'],
                            'code_blocks': problem_content['code_blocks']
                        })
        
        return problems
    
    def crawl_all(self):
        """전체 크롤링 실행"""
        post_links = self.get_post_links()
        
        if not post_links:
            print("포스트를 찾을 수 없습니다.")
            return
        
        print("\n" + "="*80)
        print("크롤링 시작!")
        print("="*80)
        
        for idx, post_info in enumerate(post_links, 1):
            print(f"\n[{idx}/{len(post_links)}] ", end="")
            
            content = self.crawl_post_content(post_info)
            
            if content:
                problems = self.extract_code_problems(content)
                
                if problems:
                    self.code_problems.extend(problems)
                    print(f"  ✓ 코드 문제 {len(problems)}개 발견!")
                    for p in problems:
                        print(f"    - {p['problem_num']}번: {p['language']}")
                else:
                    print(f"  ⚠ 코드 문제 없음")
        
        print("\n" + "="*80)
        print("크롤링 완료!")
        print("="*80)
    
    def get_statistics(self):
        """언어별 통계"""
        c_count = sum(1 for p in self.code_problems if p['language'] == 'C')
        java_count = sum(1 for p in self.code_problems if p['language'] == 'Java')
        python_count = sum(1 for p in self.code_problems if p['language'] == 'Python')
        
        return {
            'C': c_count,
            'Java': java_count,
            'Python': python_count,
            'Total': len(self.code_problems)
        }
    
    def save_to_json(self, filename='code_questions.json'):
        """JSON 저장"""
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.code_problems, f, ensure_ascii=False, indent=2)
            print(f"\n✓ JSON 파일 저장 완료: {filename}")
        except Exception as e:
            print(f"✗ JSON 저장 오류: {e}")
    
    def save_to_txt(self, filename='code_questions.txt'):
        """TXT 저장"""
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write("="*80 + "\n")
                f.write("정보처리기사 코드 문제 전체 목록\n")
                f.write("="*80 + "\n\n")
                
                for p in self.code_problems:
                    f.write("="*80 + "\n")
                    f.write(f"제목: {p['title']}\n")
                    f.write(f"URL: {p['url']}\n")
                    f.write("="*80 + "\n\n")
                    
                    f.write("[문제]\n")
                    f.write(p['question'])
                    f.write("\n\n")
                    
                    if p['code_blocks']:
                        f.write("[코드 블록]\n")
                        for idx, code in enumerate(p['code_blocks'], 1):
                            f.write(f"\n--- 코드 {idx} ---\n")
                            f.write(code)
                            f.write("\n" + "-"*40 + "\n")
                    
                    f.write("\n\n\n")
                
                # 통계
                stats = self.get_statistics()
                f.write("\n" + "="*80 + "\n")
                f.write("📊 통계\n")
                f.write("="*80 + "\n")
                f.write(f"C언어: {stats['C']}개\n")
                f.write(f"Java: {stats['Java']}개\n")
                f.write(f"Python: {stats['Python']}개\n")
                f.write(f"\n총 코드 문제: {stats['Total']}개\n")
                f.write("="*80 + "\n")
            
            print(f"✓ TXT 파일 저장 완료: {filename}")
        except Exception as e:
            print(f"✗ TXT 저장 오류: {e}")

## 4. 크롤링 실행

In [ ]:
print("="*80)
print("정보처리기사 코드 문제 크롤러 v3.4 - 진짜 최종!")
print("="*80)

category_url = "https://chobopark.tistory.com/category/Exam%20%26%20Study"

crawler = TistoryCrawler(category_url)

# 크롤링
crawler.crawl_all()

# 통계
stats = crawler.get_statistics()
print(f"\n📊 최종 통계:")
print(f"  C언어: {stats['C']}개")
print(f"  Java: {stats['Java']}개")
print(f"  Python: {stats['Python']}개")
print(f"  \n  🎯 총 코드 문제: {stats['Total']}개\n")

# 저장
crawler.save_to_json()
crawler.save_to_txt()

print("\n완료!")

## 5. 문제 샘플 확인

In [ ]:
if crawler.code_problems:
    print("\n" + "="*80)
    print("📝 문제 샘플 (첫 3개 문제)")
    print("="*80)
    
    for i, sample in enumerate(crawler.code_problems[:3], 1):
        print(f"\n[{i}] {sample['title']}")
        print(f"코드 블록: {len(sample['code_blocks'])}개")
        if sample['code_blocks']:
            print("✅ 코드 있음!")
            print(f"첫 줄: {sample['code_blocks'][0].split(chr(10))[0][:50]}...")
        else:
            print("❌ 코드 없음")
else:
    print("문제를 찾지 못했습니다.")

## 6. 파일 다운로드

In [ ]:
files.download('code_questions.json')
files.download('code_questions.txt')
print("\n파일 다운로드 완료!")

## 7. 상세 통계

In [ ]:
by_year = {}
for p in crawler.code_problems:
    key = f"{p['year']}년 {p['session']}회"
    if key not in by_year:
        by_year[key] = []
    by_year[key].append(p)

print("\n" + "="*80)
print("📊 년도별 상세 통계")
print("="*80)

for year_session in sorted(by_year.keys(), reverse=True):
    problems = by_year[year_session]
    with_code = sum(1 for p in problems if p['code_blocks'])
    print(f"\n[{year_session}]")
    print(f"  총 문제: {len(problems)}개")
    print(f"  코드 있음: {with_code}개 ✅")
    print(f"  코드 없음: {len(problems) - with_code}개 ❌")